# Example 2 - Plotting OOI Time Series Data (from NetCDF files)
*Written by Sage Lichtenwalner, Rutgers University, June 7, 2018, with help from Friedrich Knuth*

In this example we show how to programmatically download and work with NetCDF time series data from the OOI.  

We will use the data files created using Example 1, so please refer to that example first to find out how to request a specific dataset.

For reference, we will be using the [30m CTD on Global Papa Flanking Mooring B](http://ooi.visualocean.net/instruments/view/GP03FLMB-RIM01-02-CTDMOG060).

## Loading xarray
Before we get started, we first need to install and load the xarray library into Google Colab.

In [ ]:
!pip install dask
!pip install netcdf4
!pip install xarray
import xarray as xr

## The dataset 
In the previous example, we made an **Asynchronous** data request using the OOI API. We then received a two different URLs where the resulting data files can be found, a THREDDS version and a regular web site link.

Whenever you make a download request, whether you use the data portal or the API, you should also receive an email with the links in case you forgot to save them when you made the request.

In Example 1, the first URL was to the THREDDS catalog version of our dataset, which is what we'll need.  For reference, here is the [link](https://opendap.oceanobservatories.org/thredds/catalog/ooi/ooidatateam@gmail.com/20180614T193502-GP03FLMB-RIM01-02-CTDMOG060-telemetered-ctdmo_ghqr_sio_mule_instrument/catalog.html).

## Loading a single NetCDF data file

In Example 1, we requested 2 years of data (from 7/1/2015 to 7/1/2016).  This resulted in a number of files in the output directory, but only 2 key NetCDF (.nc) data files.  

*By default, the system will break up data files into individual deployments.  It will also break up the deployments if the files are larger than ~500MB.*

If we only want to load a **single NetCDF data file** (perhaps we only want one deployment or we only have one file), we could easily load it by specifying the direct link to the `.nc` file.  

To choose a single deployment file from the THREDDS catalog...
* click on the deployment/file you wish to use 
* then click on the "OPENDAP" link
* and then copy the "Data URL" from the text box.

We'll add that link here as a variable.

In [ ]:
single_file = 'https://opendap.oceanobservatories.org/thredds/dodsC/ooi/ooidatateam@gmail.com/20180615T033324-GP03FLMB-RIM01-02-CTDMOG060-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0003_GP03FLMB-RIM01-02-CTDMOG060-recovered_inst-ctdmo_ghqr_instrument_recovered_20150701T000001-20160703T183001.nc'

We can now easily load this file using xarray.

In [ ]:
# Load the data files
ds = xr.open_dataset(single_file)

# By default, OOI datasets use the 'obs' variable as the index, but time is more convenient
ds = ds.swap_dims({'obs': 'time'})

ds 

## Plotting Timeseries Data
And now we can easily use the built in matplotlib plotting routines in xarray to make a plot.

In [ ]:
ds['ctdmo_seawater_temperature'].plot();

We can also quickly make a histogram.

In [ ]:
ds['ctdmo_seawater_temperature'].plot.hist(bins=100);

As we can plot a bunch of variables at once.

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1,ax2,ax3) = plt.subplots(3,1, sharex=True, figsize=(12,7))
ds['ctdmo_seawater_temperature'].plot(ax=ax1)
ds['practical_salinity'].plot(ax=ax2)
ds['ctdmo_seawater_pressure'].plot(ax=ax3);
ax2.set_ylim(30,35);

## Loading and concatenating multiple files

In [ ]:
import requests
import os
import re
import pandas as pd

We can also concatenate multiple .nc files using xarray.  First we need to build up a list of all of the files we wish to include.  To start, let's specify the THREDDS url which contains all of our data files.

In [ ]:
url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/ooidatateam@gmail.com/20180615T033324-GP03FLMB-RIM01-02-CTDMOG060-recovered_inst-ctdmo_ghqr_instrument_recovered/catalog.html'


Next, we can use the following code to automatically find all of the available .nc files in the directory.  

In [ ]:
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = [os.path.join(tds_url, i) for i in x]
datasets

(Note, when requesting data from some instruments, you will actually get data files from multiple instruments, when they are needed to calculate derived parameters.  If this happens, you will need to modify the code above or tweak the resultant list to make sure you only include .nc files from the instrument you are interested in.)

Now we can use `open_mfdataset()` to load all files into a single xarray dataset.

In [ ]:
ds = xr.open_mfdataset(datasets)
ds = ds.swap_dims({'obs': 'time'}) # Swap the primary dimension
ds = ds.chunk({'time': 100}) # Used for optimization
ds = ds.sortby('time') # Data from different deployments can overlap so we want to sort all data by time stamp.
ds

And now a plot of this larger dataset.

In [ ]:
ds['ctdmo_seawater_temperature'].plot();

And now all the plots again.

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(3,1, sharex=True, figsize=(12,7))
ds['ctdmo_seawater_temperature'].plot(ax=ax1)
ds['practical_salinity'].plot(ax=ax2)
ds['ctdmo_seawater_pressure'].plot(ax=ax3);
ax2.set_ylim(30,35);

Let's try it again using dots instead of lines.

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(3,1, sharex=True, figsize=(12,7))
ds['ctdmo_seawater_temperature'].plot(ax=ax1,linestyle='None',marker='.',markersize=1)
ds['practical_salinity'].plot(ax=ax2,linestyle='None',marker='.',markersize=1)
ds['ctdmo_seawater_pressure'].plot(ax=ax3,linestyle='None',marker='.',markersize=1);
ax2.set_ylim(30,35);

## Identifying the Sampling Frequency
We can use the following snippet of code to find out the typical sampling frequency of the dataset.  Note, this may take a while to calculate.

In [ ]:
import pandas as pd
df = ds.to_dataframe()
res = (pd.Series(df.index[1:]) - pd.Series(df.index[:-1])).value_counts()
res

## Investigating Metadata (variable and attribute selection)
Thanks to xarray, we can easily access the global metadata as well as the metadata for individual variables.  You can refer to the full list of variables an attributes outputted above.

Here are a few examples.

In [ ]:
ds.source

In [ ]:
'%s-%s-%s' % (ds.subsite,ds.node,ds.sensor)

In [ ]:
ds['ctdmo_seawater_pressure']

In [ ]:
import textwrap
textwrap.wrap(ds['ctdmo_seawater_pressure'].comment)

In [ ]:
ds['ctdmo_seawater_pressure'].dims

In [ ]:
ds['ctdmo_seawater_pressure'].coords

In [ ]:
ds['ctdmo_seawater_pressure'].coords['pressure'].units

## Some quick statistics
We can convert the **xarray Dataset** into a **padas Dataframe** to do some quick statistical calculations.


In [ ]:
df = ds[['ctdmo_seawater_temperature','practical_salinity','ctdmo_seawater_pressure']].to_dataframe()
df = df.drop(columns=['pressure','obs']) #Drop unnecessary columns
df.head()

In [ ]:
# Prepare to be blown away...
df.describe()

## Downsampling
We can also easily calculate hourly, daily and monthly averages.  See the pandas [`resample()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html) doc for more, as well as this list of [offset options](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases).

That said, if you want to use centered averaging, moving averages, or other more complicated averaging or filtering routines using irregular intervals, you might have to roll-your-own code.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16, 6)
df['ctdmo_seawater_temperature'].plot(ax=ax,label='Raw',linestyle='None',marker='.',markersize=2)
df['ctdmo_seawater_temperature'].resample('D').mean().plot(ax=ax,label='Daily')
df['ctdmo_seawater_temperature'].resample('MS').mean().plot(ax=ax,label='Monthly',marker='d')
plt.legend();

## Creating and downloading a CSV file
Above we converted our xarray Dataset into a pandas Dataframe.  Xarray is great for loading and exporting netcdf data, while Pandas is great for doing the same with CSV.

Now that we have our data in a pandas Data frame, we can easily use the `.to_csv()` method to create a CSV file.

In [ ]:
df.to_csv('output.csv') # Create the CSV file
# ! gzip output.csv

In [ ]:
ls -l

If you're using Google Colab, these file live in the cloud, so you need to use some fancy code to save the file to your Google Drive so you can access it.  (See this [example notebook](https://colab.research.google.com/notebooks/io.ipynb) for more.)

In [ ]:
# from google.colab import files
# files.download('output.csv') # Download from Google Colab

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'test_output.csv'}) # File to be create on Drive
uploaded.SetContentFile('output.csv') # Local file to copy
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

## Bokeh Plots
Finally, let's create an interactive plot using the Bokeh library.

In [ ]:
# Import Bokeh functions
!pip install bokeh
import os
from bokeh.plotting import figure, output_file, reset_output, show, ColumnDataSource, save
from bokeh.layouts import column
from bokeh.models import BoxAnnotation
from bokeh.io import output_notebook

In [ ]:
# Let's downsample temperature a big
ds2 = ds['ctdmo_seawater_temperature'][0::10] #Start/Stop/Stride
ds2

In [ ]:
source = ColumnDataSource(
    data=dict(
        x=list(ds2.time.values),
        y=list((ds2.values*-1))
    )
)

s = figure(width=800,
           height=400,
           title='Global Papa Flanking Subsurface Mooring B Mooring Riser CTD (30 m)',
           x_axis_label='Time (GMT)',
           y_axis_label='Temperature (C)',
           x_axis_type='datetime')

# s.line('x', 'y', line_width=3, source=source)
s.circle('x', 'y', fill_color='white', size=4, source=source)

output_notebook()
show(column(s))

# output_file(os.getcwd())
# save(s, filename='pressure.html')

So that's the quick tour of some of the things you can do in Python with OOI timeseries datasets.  In Example 3, we will show you some additional tricks to use when working with vertical profiler or glider data.